# Seasonal Calculation

## Setup

### Ecocope

In [ ]:
ECOSCOPE_RAW = "https://raw.githubusercontent.com/wildlife-dynamics/ecoscope/master"

# !pip install ecoscope

In [ ]:
import os
import sys

import geopandas as gpd
import plotly.graph_objects as go

import ecoscope
import ecoscope.analysis.seasons as seasons

ecoscope.init()

### Google Drive

In [ ]:
output_dir = "Ecoscope-Outputs/landDx"

if "google.colab" in sys.modules:
    from google.colab import drive

    drive.mount("/content/drive/", force_remount=True)
    output_dir = os.path.join("/content/drive/MyDrive/", output_dir)

os.makedirs(output_dir, exist_ok=True)

### Earth Engine

In [ ]:
import ee

try:
    EE_ACCOUNT = os.getenv("EE_ACCOUNT")
    EE_PRIVATE_KEY_DATA = os.getenv("EE_PRIVATE_KEY_DATA")
    if EE_ACCOUNT and EE_PRIVATE_KEY_DATA:
        ee.Initialize(credentials=ee.ServiceAccountCredentials(EE_ACCOUNT, key_data=EE_PRIVATE_KEY_DATA))
    else:
        ee.Initialize()

except ee.EEException:
    ee.Authenticate()
    ee.Initialize()

## Download AOI

In [ ]:
ecoscope.io.download_file(
    f"{ECOSCOPE_RAW}/tests/sample_data/vector/maec_4zones_UTM36S.gpkg",
    os.path.join(output_dir, "maec_4zones_UTM36S.gpkg"),
)

aoi = gpd.read_file(os.path.join(output_dir, "maec_4zones_UTM36S.gpkg")).to_crs(4326)
aoi.explore()

## Merge to Larger Polygon

In [ ]:
aoi = aoi.dissolve()
aoi = aoi.iloc[0]["geometry"]

## Calculate Seasons

In [ ]:
# Extract the standardized NDVI ndvi_vals within the AOI
ndvi_vals = seasons.std_ndvi_vals(aoi, start="2010-01-01", end="2022-06-18")

# Calculate the seasonal transition point
cuts = seasons.val_cuts(ndvi_vals, 2)

# Determine the seasonal time windows
windows = seasons.seasonal_windows(ndvi_vals, cuts, season_labels=["dry", "wet"])

In [ ]:
windows

## Export

In [ ]:
windows.to_csv(os.path.join(output_dir, "season_time_windows.csv"), date_format="%Y-%m-%d %H:%M:%S", index=True)

## Visualize

In [ ]:
fig = ecoscope.plotting.plot.add_seasons(go.Figure(), windows)

# Export image
fig.write_image(os.path.join(output_dir, "seasonal_timeline.png"))

fig

## Visualize NDVI Values and Seasonal Transition

In [ ]:
# Plot the NDVI Distribution and Seasonal Transition
fig = ecoscope.plotting.plot.plot_seasonal_dist(
    ndvi_vals["NDVI"], cuts, output_file=os.path.join(output_dir, "seasonal_transition.png")
)
fig.show()